In [25]:
import numpy as np
import pandas as pd
import tokenizers # 토큰화(tokenization)
import gensim # 임베딩(embedding) : 토큰을 숫자(벡터/좌표)로 변환
from tqdm import tqdm

### 1. 데이터 기초 전처리 및 사전 기반 알고리즘의 활용

In [2]:
np.random.seed(0)
moviedf = pd.read_csv('./data/naver_movie.csv', index_col=0).sample(500)
shoppingdf = pd.read_csv('./data/naver_shopping.csv', index_col=0).sample(500)
beepdf = pd.read_csv('./data/beep_dataset.csv', index_col=0).sample(500)
jobplanetdf = pd.read_csv('./data/jobplanet_review.csv', index_col=0)

moviedf.columns = ['text', 'label']
shoppingdf.columns = ['text', 'label']

In [3]:
textdf =\
pd.concat([moviedf, shoppingdf, beepdf,
           jobplanetdf[jobplanetdf['label']==1][['good', 'label']].rename({'good':'text'}, axis=1),
           jobplanetdf[jobplanetdf['label']==0][['bad', 'label']].rename({'bad':'text'}, axis=1)],
          axis=0, ignore_index=True)
textdf['label'] = textdf['label'].replace({0:1, 1:0})

In [15]:
bad_words = ['나쁜', '구린', '썩은', '별로', '최악', '끔찍', 
             '지루', '잠', '환불', '늑장', '늦장', '꼰대', '정치질', '억압'
             '하향', '수직']
textdf['pred'] = textdf['text'].str.contains('|'.join(bad_words)).astype(int)
textdf['correct'] = (textdf['label']==textdf['pred']).astype(int)
np.mean(textdf['correct'])

0.5465084184377588

### 2. 토큰화

In [18]:
mobiledf = pd.read_csv('./data/mobilecarrier_news.csv', index_col=0)
welfaredf = pd.read_csv('./data/welfare_news.csv', index_col=0)

In [36]:
corpus = pd.concat([textdf['text'], 
                    welfaredf['title'], welfaredf['content'], 
                    mobiledf['title'].drop_duplicates()], axis=0, ignore_index=True)
corpus = corpus.str.replace('\[(.*?)\]', '')
corpus = corpus.str.split().str.join(' ')

In [48]:
from tokenizers import SentencePieceBPETokenizer
bpe_tokenizer = SentencePieceBPETokenizer()
bpe_tokenizer.train_from_iterator(corpus)

In [50]:
bpe_tokenizer.encode('안녕하세요저는방송사에서PD로일을하고있습니다').tokens

['▁안녕', '하세요', '저', '는', '방송', '사에서', 'P', 'D', '로', '일을', '하고', '있', '습니다']

In [53]:
from tqdm import tqdm

In [54]:
token_corpus = []
for text in tqdm(corpus) :
    token_corpus.append(bpe_tokenizer.encode(text).tokens)
    
token_corpus = pd.Series(token_corpus)

100%|██████████████████████████████████████████████████████████████████████████| 15588/15588 [00:06<00:00, 2595.41it/s]


In [55]:
token_corpus = pd.Series([bpe_tokenizer.encode(text).tokens for text in tqdm(corpus)])

100%|██████████████████████████████████████████████████████████████████████████| 15588/15588 [00:05<00:00, 2606.79it/s]


In [52]:
token_corpus

0        [▁가르, 시아, 를, ▁알게, ▁해준, ▁영화, .., 마, 지막, ▁장면, 도,...
1        [▁이영, 화를, 보면서, ▁아프, 겠다, 라는, 생각, 이, 들, 엇, 다, ▁ᄏ...
2                          [▁이, ▁영화를, ▁보고, ▁암, 이, ▁나, 았다.]
3        [▁민, 들, 레, 부모가, ▁전하는, ▁아, 가, 페, 적인, 사랑, ..., 가...
4          [▁어린이집, ▁재, 롱, 잔, 치를, ▁만, 원, 주고, ▁보는, ▁기분, 이라면]
                               ...                        
15583            [▁SKT,, ▁3년, ▁연속, ▁', 최고, ▁이통사, ', ▁영, 예]
15584                [▁KT,, ▁AI, ▁코딩, ▁초등, 생, ▁꿈, 나무, ▁시상]
15585    [▁', 딸, ▁KT, ▁특별, 채용, ', ▁김성, 태, ▁사퇴, 에, .., 민...
15586    [▁SKT,, ▁내일, ▁분할, ▁후, ▁재, 상장, .., 주가, ▁날아, 오를, 까]
15587     [▁LG유플러스,, ▁인공지능, ▁관리, ▁'스마트, ▁양, 계, 장, ', ▁만든다]
Length: 15588, dtype: object

### 3. 임베딩

### 4. 머신러닝